# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

#     Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.

#     Create a heat map that displays the humidity for every city from the part I of the homework.
#     Narrow down the DataFrame to find your ideal weather condition. For example: 
# ** A max temperature lower than 27 degrees but higher than 21. 
# ** Wind speed less than 15 kph. 
# ** Zero cloudiness. 
# ** Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. 

# ** Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

#     Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#     Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# As final considerations:

#     You must complete your analysis using a Jupyter notebook.
#     You must use the Matplotlib or Pandas plotting libraries.
#     For Part I, you must include a written description of three observable trends based on the data.
#     For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
#     You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
#     For max intensity in the heat map, try setting it to the highest humidity found in the data set.


In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

pd.options.mode.chained_assignment = None  
# default='warn'

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
new_cities_df = pd.read_csv("../output_data/carlys_cities.csv")
new_cities_df

,City Name,Country,Latitude,Longitude,Maximum Temperature (d/c),Humidity (%),Cloudiness (%),Wind Speed (km/h),Date
0,Punta Arenas,CL,-53.1500,-70.9167,1.06,86,20,2.06,1651841108
1,Chui,UY,-33.6971,-53.4616,16.19,67,0,2.31,1651841108
2,Suffolk,US,40.8668,-72.7996,13.95,88,100,2.57,1651841108
3,Puerto Escondido,MX,15.8500,-97.0667,23.46,83,0,3.09,1651841109
4,Yellowknife,CA,62.4560,-114.3525,6.29,70,75,3.60,1651841109
...,...,...,...,...,...,...,...,...,...
511,Mugeni,RO,46.2500,25.2167,20.20,43,9,2.08,1651841354
512,Weymouth,US,42.2209,-70.9398,15.02,54,100,2.06,1651841355
513,Mt. Pleasant,US,32.7941,-79.8626,26.14,80,0,3.60,1651841355
514,Ambulu,ID,-8.3450,113.6058,25.50,91,45,1.68,1651841355


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [13]:
# plot heatmap
heatmap_layer = gmaps.heatmap_layer(new_cities_df[['Latitude', 'Longitude']].values, weights=new_cities_df['Humidity (%)'].values, dissipating=True)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

In [14]:
# Carlys Personal Criteria
# Humidity less than 45%
# max temperature lower than 28 degrees but higher than 18.
# wind speed less than 15 kph. 
# ** Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. 

holiday_destinations_df = new_cities_df[(new_cities_df["Humidity (%)"] < 40) &
                                            (new_cities_df["Wind Speed (km/h)"] < 15) & 
                                            (new_cities_df["Maximum Temperature (d/c)"] < 27) & 
                                            (new_cities_df["Maximum Temperature (d/c)"] > 20)]
holiday_destinations_df

,City Name,Country,Latitude,Longitude,Maximum Temperature (d/c),Humidity (%),Cloudiness (%),Wind Speed (km/h),Date
32,Lady Frere,ZA,-31.7031,27.2329,25.17,20,3,5.32,1651841119
70,Port Hedland,AU,-20.3167,118.5667,25.03,36,0,3.60,1651841136
87,Korla,CN,41.7597,86.1469,25.80,31,100,4.31,1651841143
128,Manavgat,TR,36.7867,31.4431,23.07,32,4,9.48,1651841167
132,Salamiyah,SY,35.0113,37.0532,20.04,27,43,11.03,1651841168
141,Karratha,AU,-20.7377,116.8463,26.93,33,0,5.60,1651841178
153,Zaragoza,ES,41.6561,-0.8773,21.36,38,20,10.80,1651841181
161,Faya,SA,18.3851,42.4509,26.06,37,13,6.29,1651841184
188,Abu Samrah,SY,35.3029,37.1841,20.40,26,95,9.64,1651841198
200,Shambu,ET,9.5667,37.1000,25.29,17,76,2.25,1651841201


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Cretae a new column to populate the hotel name into

holiday_destinations_df = holiday_destinations_df
holiday_destinations_df["Hotel Name"] = " "
holiday_destinations_df

,City Name,Country,Latitude,Longitude,Maximum Temperature (d/c),Humidity (%),Cloudiness (%),Wind Speed (km/h),Date,Hotel Name
32,Lady Frere,ZA,-31.7031,27.2329,25.17,20,3,5.32,1651841119,
70,Port Hedland,AU,-20.3167,118.5667,25.03,36,0,3.60,1651841136,
87,Korla,CN,41.7597,86.1469,25.80,31,100,4.31,1651841143,
128,Manavgat,TR,36.7867,31.4431,23.07,32,4,9.48,1651841167,
132,Salamiyah,SY,35.0113,37.0532,20.04,27,43,11.03,1651841168,
141,Karratha,AU,-20.7377,116.8463,26.93,33,0,5.60,1651841178,
153,Zaragoza,ES,41.6561,-0.8773,21.36,38,20,10.80,1651841181,
161,Faya,SA,18.3851,42.4509,26.06,37,13,6.29,1651841184,
188,Abu Samrah,SY,35.3029,37.1841,20.40,26,95,9.64,1651841198,
200,Shambu,ET,9.5667,37.1000,25.29,17,76,2.25,1651841201,


In [16]:
#holiday_destinations_df

# find nearest hotel

def find_hotel(lat, lng, radius):
    """
    Finds the nearest hotel to the given coordinates.
    """
    # set up a parameters dictionary
    params = {"location": f"{lat},{lng}",
                "radius": radius,
                "type": "lodging",
                "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    # print(json.dumps(places_data, indent=4, sort_keys=True))

    # Print the name and address of the first hotel that appears
    hotel_name = "None"
    if len(places_data["results"]) > 0:
        hotel_name = places_data["results"][0]["name"]
    return hotel_name

In [21]:
# find hotel name for each city and Add it to the dataframe
holiday_destinations_df["Hotel Name"] = holiday_destinations_df.apply(lambda row: find_hotel(row["Latitude"], row["Longitude"], radius=5000), axis=1)

In [23]:
# some cities don't have a hotel in the indicated radius, so these cities will need to be dropped
holiday_destinations_df = holiday_destinations_df.loc[holiday_destinations_df["Hotel Name"] != "None"]

# Display final holiday destinations dataframe
holiday_destinations_df

,City Name,Country,Latitude,Longitude,Maximum Temperature (d/c),Humidity (%),Cloudiness (%),Wind Speed (km/h),Date,Hotel Name
32,Lady Frere,ZA,-31.7031,27.2329,25.17,20,3,5.32,1651841119,Goshen BnB
70,Port Hedland,AU,-20.3167,118.5667,25.03,36,0,3.60,1651841136,The Esplanade Hotel
87,Korla,CN,41.7597,86.1469,25.80,31,100,4.31,1651841143,Gulisitan Hotel
128,Manavgat,TR,36.7867,31.4431,23.07,32,4,9.48,1651841167,Lake & River Side Hotel & Spa
132,Salamiyah,SY,35.0113,37.0532,20.04,27,43,11.03,1651841168,Gondolas Forum grandchildren and grandparents
141,Karratha,AU,-20.7377,116.8463,26.93,33,0,5.60,1651841178,ibis Styles Karratha
153,Zaragoza,ES,41.6561,-0.8773,21.36,38,20,10.80,1651841181,INNSiDE Zaragoza
161,Faya,SA,18.3851,42.4509,26.06,37,13,6.29,1651841184,املاك خالد الشعشوع
200,Shambu,ET,9.5667,37.1000,25.29,17,76,2.25,1651841201,Wabe Hotel
245,Vryburg,ZA,-26.9566,24.7284,23.93,22,0,2.14,1651841223,The Times Premier Lodge


In [25]:
# TEMPLATE TO PLOT THE PIN LAYER ON THE HEATMAP WHICH INCLUDES THE HOTEL NAME, CITY AND COUNTRY
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name: {Hotel Name}
City: {City Name}
Country: {Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in holiday_destinations_df.iterrows()]
locations = holiday_destinations_df[["Latitude", "Longitude"]]

In [26]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

gmaps.configure(api_key=g_key) 
fig = gmaps.figure()
# plot heatmap
heatmap_layer = gmaps.heatmap_layer(holiday_destinations_df[['Latitude', 'Longitude']].values, weights=holiday_destinations_df['Humidity (%)'].values, dissipating=True)

fig.add_layer(heatmap_layer)
# add hotel name, city, and country to the heatmap
fig.add_layer(gmaps.marker_layer(holiday_destinations_df[['Latitude', 'Longitude']].values, hotel_info))

fig

Figure(layout=FigureLayout(height='420px'))